# Nonnegative Matrix Factorisation for Audio Applications
## Dan Jacobellis, Tyler Masthay
![](VWH.png)

## Nonnegative Matrix Factorisation

&nbsp;

$$ \bf V \approx \hat {\bf V} = \bf W \bf H$$

$$\matrix{
\bf W \in \Bbb R^{m \times k} & \bf W \geq 0 \\
\bf H \in \Bbb R^{k \times n} & \bf H \geq 0}$$


- Unsupervised learning (think SVD)
- Number of components $k$ is a parameter
- $\bf V$ is $m \times n$
- Setting $k \lt m$ is a form of lossy compression

## Audio Example

## $\bf V =\left| CQT(x) \right|=$

![](V.png)

*Time-frequency representation of 'Korobeiniki' played on piano. The frequency is on a logarithmic scale.*

## Audio Example

## $\bf V =\left| CQT(x) \right|=$

![](V_poly.png)

*Time-frequency representation of 'Korobeiniki' (polyphonic).*

The phase information is less important to our perception of the audio than the magnitude, so we take the magnitude of the time-frequency decomposition. This is why we need nonnegative. There is no such thing as a negative note or a negative speech.

Notice that even though the recording consists of a single instrument (piano) playing a monophonic melody, the harmonics of each note create replicas of the actual melody at integer multiples of the fundamental frequency. The complexity that arises from mixing multiple intruments or adding any amount of polyphony makes direct analysis of the time-frequency image intractable for most tasks.

## Audio Example

![](VWH.png)

When $k \lt m$ , and we search for a factorization that approximately matches $\bf V$, we perform unsupervised learning. $\bf W$ contains a learned dictionary and $\bf H$ contains a representation of $\bf V$ in terms of this dictionary. 

This makes many forms of audio analysis more tractable, especially source separation and music transcription.

## Algorithm: Multiplicative Update

&nbsp;

* Initialize $\bf W$ and $\bf H$ with non-negative values
* Iteratively update $\bf W$ and $\bf H$ using the following rules: ($n$ here is the iteration)

$$
{\bf H}_{[i,j]}^{n+1}\leftarrow {\bf H}_{[i,j]}^{n} \odot \frac
{\left( ({\bf W}^{n})^\top \bf V \right)_{[i,j]}}
{\left( ({\bf W}^n)^\top {\bf W}^n {\bf H}^n \right)_{[i,j]}}
$$

$$
{\bf W}_{[i,j]}^{n+1}\leftarrow {\bf W}_{[i,j]}^{n} \odot \frac
{\left( {\bf V} ({\bf H}^{n+1})^\top \right)_{[i,j]}}
{\left( {\bf W}^n {\bf H}^{n+1} ({\bf H}^{n+1})^\top \right)_{[i,j]}}
$$

$\odot$ and division are element-wise.

ref. Lee, D.D., Seung, H.S., 2001. Algorithms for Non-negative Matrix Factorization, in: Advances in Neural Information Processing Systems 13. MIT Press, pp. 556–562.

## Algorithm: Multiplicative Update

![](DAG1.png)

Must be careful with order of operations, otherwise the matrix size will be increased unnecessarily.

We want to exploit all of these embarassingly parallel steps.

## Parallelization on GPU : Motivation

- Matricies remain stationary in memory
- Matrix multiplies have high computational intensity compared to memory
- Single precision is sufficient
- Would like to use consumer hardware

![](mobileGPU.png)

Since we have good memory locality and high computational intensity (high flops to mops) the algorithm is a good fit for GPU.

One of the typical drawbacks to using GPU is poor double precision, but most audio is 16 bit fixed point, so single is sufficient.

Applications become much more useful if we can use consumer or even mobile hardware.

## GPU Implementation

- Used Julia bindings to CUDA (CURAND, CUBLAS)
- Compiler allows easily mapping high level syntax to GPU
- Compiler finds best way to send data back and forth between device and host
![](PTX.png)

ref. https://github.com/JuliaGPU/CUDAnative.jl

## GPU Implementation
- Must be careful and explicit with types and constants
- Otherwise, compiler will think you want to move all of the data off of the GPU and back
![](nvprof.png)

# GPU Performance
 ![](gpu_runtime.png)
 
 - Runtime is relatively flat as the number of components increases up to a point
 - We can't fit everying in GPU memory after the number of components reaches ~128
 - After this point, we are forced to send data between CPU and GPU *every* iteration
 - Fortunately, the behavior as a function of the audio length is well-behaved

# Compilation Performance

&nbsp;

![](Compilation.png)
 
 - Left figure shows orders of magnitude slowdown on the initial compilation of code for a variety of commonly used functions
 - Right figure shows speedup for subsequent compilations of code
 
 ref. Besard et. al. *Effective Extensible Programming: Unleashing Julia on GPUs*.

# Lines of Code

&nbsp;

![](Rodinia.png)
 
 - Lines of host and device code for Rodinia benchmarks. Comparison between CUDA C and Julia implementations.
 - On average, device code reduced by 8 percent and host code by 38 percent.

# GPU Performance
 ![](gpu_runtime.png)
 
 - Runtime is relatively flat as the number of components increases up to a point
 - We can't fit everying in GPU memory after the number of components reaches ~128
 - After this point, we are forced to send data between CPU and GPU *every* iteration
 - Fortunately, the behavior as a function of the audio length is well-behaved

## CPU Performance

![](cpu_runtime.png)

- Scikit-learn implementation would require many CPU cores to match GPU runtime

![](cpu_gpu_table.png)

Runtime (in seconds) vs number of components $(k)$ vs length of processed audio ($n$) 

## Demonstration

## $\bf V =\left| CQT(x) \right|=$

![](V.png)

*Time-frequency representation of 'Korobeiniki' (monophonic).*

## Demonstration
![](W.png)

*$\bf W$ matrix arranged by fundamental frequency*

## Demonstration
![](H.png)

*$\bf H$ matrix collapsed into midi note bins*

## Demonstration

## $\bf V =\left| CQT(x) \right|=$

![](V_poly.png)

*Time-frequency representation of 'Korobeiniki' (polyphonic).*

## Demonstration
![](W_poly.png)

*$\bf W$ matrix arranged by fundamental frequency*

## Demonstration
![](H_poly.png)

*$\bf H$ matrix collapsed into midi note bins*

## Backup
![](TFP.png)